### Урок 7. Кейс 2. Типы моделей для задачи тарификации

### -- Автор: Шенк Евгений Станиславович

### Домашнее задание (часть 1)

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать eda (как на занятии 6 с примером для оттока)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

Выполнение:
    1. Взят датасет https://archive.ics.uci.edu/ml/machine-learning-databases/00222/  
        - там архив поэтому подгружается из папки с нотбуком
    2. по аналогии с 6 уроком проведина EDA
    3. Проведен feature engineering: созданы дамми переменные, выделены числовые и категориальные переменные, группы по целевым переменным.
    4. использован xgBoost
    5. выполнено
    6. выполнено
    7. Качество (f1 score) оказалось ниже у RNS, хотя и не сильно.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import plotly.graph_objs as go#visualization
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization

In [2]:
# https://archive.ics.uci.edu/ml/machine-learning-databases/00222/

path = "bank.csv"
data = pd.read_csv(path, delimiter=';')

In [3]:
print(data.shape)
data.head(3)

(4521, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no


In [4]:
data.describe()

,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


In [6]:
data['y'].groupby(data['y']).count()

y
no     4000
yes     521
Name: y, dtype: int64

In [7]:
#categorical columns
cat_cols = []
for col in data.columns:
    if data[col].dtype == 'O':
        cat_cols.append(col)
        print(f"{col}: {data[col].unique()}")

job: ['unemployed' 'services' 'management' 'blue-collar' 'self-employed'
 'technician' 'entrepreneur' 'admin.' 'student' 'housemaid' 'retired'
 'unknown']
marital: ['married' 'single' 'divorced']
education: ['primary' 'secondary' 'tertiary' 'unknown']
default: ['no' 'yes']
housing: ['no' 'yes']
loan: ['no' 'yes']
contact: ['cellular' 'unknown' 'telephone']
month: ['oct' 'may' 'apr' 'jun' 'feb' 'aug' 'jan' 'jul' 'nov' 'sep' 'mar' 'dec']
poutcome: ['unknown' 'failure' 'other' 'success']
y: ['no' 'yes']


In [8]:
#Target columns
target_col = ["y"]
#numerical columns
num_cols   = [x for x in data.columns if x not in cat_cols + target_col]
#Binary columns with 2 values
bin_cols   = data.nunique()[data.nunique() == 2].keys().tolist()
#Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

#Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    data[i] = le.fit_transform(data[i])
    
#Duplicating columns for multi value columns
data = pd.get_dummies(data = data, columns = multi_cols)

#Scaling Numerical columns
std = StandardScaler()
scaled = std.fit_transform(data[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

#dropping original values merging scaled values for numerical columns
df_data_og = data.copy()
data = data.drop(columns = num_cols,axis = 1)
data = data.merge(scaled,left_index=True,right_index=True,how = "left")

#Separating 'yes' and 'no'  customers
target_yes = data[data["y"] == 1]
target_no = data[data["y"] == 0]

#Separating catagorical and numerical columns
target_col = ["y"]
cat_cols   = data.nunique()[data.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in data.columns if x not in cat_cols + target_col]

#Move target to last position
target = data["y"]
data = data.drop(columns = 'y', axis=1)
data["y"] = target

In [9]:
def plot_pie(column) :
    
    trace1 = go.Pie(values  = target_yes[column].value_counts().values.tolist(),
                    labels  = target_yes[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    domain  = dict(x = [0,.48]),
                    name    = "Yes",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    hole    = .6
                   )
    trace2 = go.Pie(values  = target_no[column].value_counts().values.tolist(),
                    labels  = target_no[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    domain  = dict(x = [.52,1]),
                    hole    = .6,
                    name    = "No" 
                   )


    layout = go.Layout(dict(title = column + " distribution in customer attrition ",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            annotations = [dict(text = "Yes",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .15, y = .5),
                                           dict(text = "No",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .88,y = .5
                                               )
                                          ]
                           )
                      )
    data = [trace1,trace2]
    fig  = go.Figure(data = data,layout = layout)
    py.iplot(fig)

In [10]:
#for all categorical columns plot pie
for i in cat_cols :
    plot_pie(i)

### обучаем классификатор XGBClassifier

In [11]:
from sklearn.model_selection import train_test_split

x_data = data.drop(columns = target_col, axis = 1)
y_data = data[target_col]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [12]:
import xgboost as xgb

model = xgb.XGBClassifier(learning_rate = 0.01,
                            max_depth = 10,
                            n_estimators = 200,
                            subsample= 1,
                            reg_alpha = 0,
                            reg_lambda = 0,
                            n_jobs=-1,
                            random_state=123)

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

C:\Users\Eugene\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [13]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 41.21%
roc: 64.66%
recall: 32.69%
precision: 55.74%


### Test the PU learning approach

* Keep aside 25% of the positives -- they will be the only labeled samples

In [14]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data[target_col].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 131/521 as positives and unlabeling the rest


* Create the target col 'class_test' that will be 1 for postive and -1 for unlabebed¶

In [15]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    4390
 1     131
Name: class_test, dtype: int64


* We now have just 153 positive samples labeled as 1 in the 'class_test' col while the rest is unlabeled as -1.

* Recall that col 4 still holds the actual label

In [16]:
mod_data.head(10)

,default,housing,loan,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,...,poutcome_unknown,age,balance,day,duration,campaign,pdays,previous,y,class_test
0,0,0,0,0,0,0,0,0,0,0,...,1,-1.056270,0.121072,0.374052,-0.711861,-0.576829,-0.407218,-0.320413,0,-1
1,0,1,1,0,0,0,0,0,0,0,...,0,-0.772583,1.118644,-0.596026,-0.169194,-0.576829,2.989044,2.041734,0,-1
2,0,1,0,0,0,0,0,1,0,0,...,0,-0.583458,-0.024144,0.010273,-0.303898,-0.576829,2.899143,0.270124,0,-1
3,0,1,1,0,0,0,0,1,0,0,...,1,-1.056270,0.017726,-1.566105,-0.250017,0.387967,-0.407218,-0.320413,0,-1
4,0,1,0,0,1,0,0,0,0,0,...,1,1.686036,-0.472753,-1.323585,-0.146102,-0.576829,-0.407218,-0.320413,0,-1
5,0,0,0,0,0,0,0,1,0,0,...,0,-0.583458,-0.224523,0.859091,-0.473241,-0.255231,1.360836,1.451197,0,-1
6,0,1,0,0,0,0,0,0,0,1,...,0,-0.488896,-0.370736,-0.232247,0.296499,-0.576829,2.899143,0.860660,0,-1
7,0,1,0,0,0,0,0,0,0,0,...,1,-0.205209,-0.423904,-1.202326,-0.434754,-0.255231,-0.407218,-0.320413,0,-1
8,0,1,0,0,0,1,0,0,0,0,...,1,-0.016085,-0.399314,-0.232247,-0.796532,-0.255231,-0.407218,-0.320413,0,-1
9,0,1,1,0,0,0,0,0,0,0,...,0,0.173040,-0.501996,0.131532,0.188735,-0.576829,1.071155,0.860660,0,-1


Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [17]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### random negative sampling

In [18]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(131, 50) (131, 50)


In [19]:
model = xgb.XGBClassifier(learning_rate = 0.01,
                            max_depth = 10,
                            n_estimators = 200,
                            subsample= 1,
                            reg_alpha = 0,
                            reg_lambda = 0,
                            n_jobs=-1,
                            random_state=123)

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 36.57%
roc: 77.51%
recall: 80.16%
precision: 23.69%


In [20]:
from sklearn.metrics import f1_score

#gives model report in dataframe
def model_report(model,training_x,testing_x,training_y,testing_y,name) :
    model.fit(training_x,training_y)
    predictions  = model.predict(testing_x)
    accuracy     = accuracy_score(testing_y,predictions)
    recallscore  = recall_score(testing_y,predictions)
    precision    = precision_score(testing_y,predictions)
    roc_auc      = roc_auc_score(testing_y,predictions)
    f1score      = f1_score(testing_y,predictions) 
    
    df = pd.DataFrame({"Model"           : [name],
                       "Accuracy_score"  : [accuracy],
                       "Recall_score"    : [recallscore],
                       "Precision"       : [precision],
                       "f1_score"        : [f1score],
                       "Area_under_curve": [roc_auc],
                      })
    return df

In [21]:
xgboost = xgb.XGBClassifier(learning_rate = 0.01,
                            max_depth = 10,
                            n_estimators = 200,
                            subsample= 1,
                            reg_alpha = 0,
                            reg_lambda = 0,
                            n_jobs=-1,
                            random_state=123)
model1 = model_report(xgboost, x_train, x_test, y_train, y_test,
                      "XGBoost Classifier")

xgboost = xgb.XGBClassifier(learning_rate = 0.01,
                            max_depth = 10,
                            n_estimators = 200,
                            subsample= 1,
                            reg_alpha = 0,
                            reg_lambda = 0,
                            n_jobs=-1,
                            random_state=123)
model2 = model_report(xgboost,sample_train.iloc[:,:-2].values,sample_test.iloc[:,:-2].values,sample_train.iloc[:,-2].values,sample_test.iloc[:,-2].values,
                      "XGBoost Classifier (RNS)")

#concat all models
model_performances = pd.concat([model1,model2],axis = 0).reset_index()

model_performances = model_performances.drop(columns = "index",axis =1)

model_performances

C:\Users\Eugene\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



,Model,Accuracy_score,Recall_score,Precision,f1_score,Area_under_curve
0,XGBoost Classifier,0.892818,0.326923,0.557377,0.412121,0.646608
1,XGBoost Classifier (RNS),0.753228,0.801587,0.236904,0.365721,0.775053
